## High-precision Computation in Python

Typically, you can use python or any language for basic calculations without worrying about how many bits are used to represent your numbers.  But occasionally, you may need to make very high precision computations in python.  I've written this tutorial to help you prepare for this need.

In my case, high-precision calculation has come up in the context of modeling the motion of bodies in the Solar System.  That requires the integration of gravitational equations of motion over many decades.  This requires some understanding of things like machine precision and round-off error, that I otherwise don't need to worry about.

By the way, a wonderful resource for precision computing is the Numerial Recipes textbook.  I learned a great deal from that reference, and borrowed heavily from it to make this notebook.

# Machine Accuracy

First of all, it's important to know that python cannot keep track of arbitrarily small values.
For example, if you create a variable and assign it the value 1.0, and then add to it 1e-30, the number in memory is unchanged.

In [22]:
myval = 1.0
print myval
myval += 1e-30
print myval

1.0
1.0


By default, python stores numbers like 1.0 as 64-bit floating point values (so-called `float64`).  You can find out what the machine accuracy of `float64` is using the `np.finfo()` function in `numpy`.

In [31]:
import numpy as np
print np.finfo(1.0)
print "Number of bits: ", np.finfo(np.float64).bits

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
---------------------------------------------------------------

Number of bits:  64


So if we add 1e-15 or larger, then python will "notice"

In [9]:
myval = 1.0
print myval
myval += 2e-15
print myval

1.0
1.0


Wait, what's going on here?  Why do I still see 1.0 and not 1.000000000000002?  Well, python doesn't print values at full precision.  You need to ask it to do so.

In [82]:
digitlabel = 'X.1...5....0....5..8.0\nX..........1....1..1.2'

print '{0:.16f}'.format(1)
print '{0:.16f}'.format(1+2e-15)
print '{0:.16f}'.format(1+1e-15)
print '{!r}'.format(1+1e-15)       # {!r} means first, call repr() on the value, then print it out
print digitlabel

1.0000000000000000
1.0000000000000020
1.0000000000000011
1.000000000000001
X.1...5....0....5..8.0
X..........1....1..1.2


There, that's much better!  Hmm, maybe the original attempt of adding 1e-30 was actually fine...  Let's check:

In [83]:
print '{0:.32f}'.format(1)
print '{0:.32f}'.format(1+1e-30)
print '{!r}'.format(1+1e-30)

1.00000000000000000000000000000000
1.00000000000000000000000000000000
1.0


Nope!  Python really can't tell the difference between 1.0 and (1.0+1e-30).  If you'd like to know more details about *why* python can't tell the difference, please see "Floating-point representation" below.

## longdouble gives higher precision
If you need more than 64 bits, then python also has something called `longdouble` that uses 80 bits (well, the actual number of bits used depends on the platform. see: https://docs.scipy.org/doc/numpy-dev/user/basics.types.html

In [32]:
print np.finfo(np.longdouble)
print "Number of bits: ", np.finfo(np.longdouble).bits

Machine parameters for float128
---------------------------------------------------------------
precision =  18   resolution = 1e-18
machep =    -63   eps =        1.08420217249e-19
negep =     -64   epsneg =     5.42101086243e-20
minexp = -16382   tiny =       3.36210314311e-4932
maxexp =  16384   max =        1.18973149536e+4932
nexp =       15   min =        -max
---------------------------------------------------------------

Number of bits:  128


Hmm, this seems to suggest that `longdouble` has 128 bits...  But that is actually a half-truth.  python does use 128 bits of memory to store the value, BUT, only 80 bits contain values, the rest is just zero-padding.

## We can verify that a longdouble will "notice" if 1e-18 has been added (but will not notice if 1e-19 is added)

In [88]:
myld = np.longdouble(0.1)
myd  = np.float64(0.1)
print '{!r}'.format(myld)
print '{!r}'.format(myld+1e-18)
print '{!r}'.format(myld+1e-19)
print '{!r}'.format(myd)          
print '{!r}'.format(myd+1e-18)   # a "regular" double doesn't notice the 1e-18
print digitlabel

0.10000000000000000555
0.10000000000000000655
0.10000000000000000565
0.10000000000000001
0.10000000000000001
X.1...5....0....5..8.0
X..........1....1..1.2


## Maintaining precision during calculations
If we have a `longdouble`, and then do an operation like `sin()` or `sqrt()`, what precision do we end up with?

In [96]:
myld = np.longdouble(4.1)
print '{!r}'.format(myld)
print '{!r}'.format(myld+1e-18)
print digitlabel
print '{!r}'.format(np.sqrt(myld))

4.0999999999999996447
4.0999999999999996456
X.1...5....0....5..8.0
X..........1....1..1.2
2.0248456731316586057


WTF: the representation of 4.1 isn't even good to 18 digits... How can that be for an 80-bit longdouble?

Well, what is the right answer?  We can use mpmath to tell us...

In [41]:
import mpmath as mp
mp.mp.dps = 40   # 40 digits of precision
print(mp.mp)
mp.sqrt(mp.mpf(4.1))

Mpmath settings:
  mp.prec = 136               [default: 53]
  mp.dps = 40                 [default: 15]
  mp.trap_complex = False     [default: False]


mpf('2.024845673131658605596679004662654403763166')

Soooo, the "true" answer (to 40 digits anyway), is:  
`2.024845673131658605596679004662654403763166`  
using `longdouble`, we find:  
`2.0248456731316586057`  
`--123456789012345678^`
       
which disagrees with truth at the 19th decimal place.

How about np.sin()?

In [101]:
myld = np.longdouble(0.1)
print '{!r}'.format(np.sin(myld))
mp.sin(mp.mpf(0.1))

0.099833416646828157833


mpf('0.09983341664682815783019686785861666773596606')

So the "true" answer (to 40 digits) is:  
    `0.09983341664682815783019686785861666773596606`  
    using `longdouble` we find:  
    `0.099833416646828157833`  
    `--12345678901234567890^`  
Hmm, how can the output of `np.sin()` agree to more digits than the input did?
    

# APPENDICES

## Floating-Point Representation
From Numerical Recipes:

"In a floating-point representation, a number is represented internally by a sign bit $S$ (interpreted as plus or minus), an exact integer exponent $E$, and an exactly represented binary mantissa $M$. Taken together these represent the number:
$$ S \times M \times 2^{E-e} $$ 
where $e$ is the *bias* of the exponent, a fixed integer constant for any given machine and representation."

For 64-bit `double` values, the exponent takes 11 bits (with $e=1023$), and the mantissa takes 52 bits.  The final bit is the sign bit $S$ (0=positive, 1=negative).  For example, the binary representations of the decimal values 2.0 and 6.5 are:
\begin{align*}
0\,10000000000\,0000\,\mbox{(+ 48 more zeros)} &= +1 \times 2^{1024-1023} \times 1.0_2 = 2. \\
0\,10000000001\,1010\,\mbox{(+ 48 more zeros)} &= +1 \times 2^{1025-1023} \times 1.1010_2 = 6.5
\end{align*}
note, the way to read the last item in the row (e.g. $1.1010_2$ for the decimal 6.5 case) is that each digit after the decimal represents 2 to some negative power.  The first 1 is $2^{-1}$.  Then there are zero $2^{-2}$ but one $2^{-3}$.  So we have $2^{-1} + 2^{-3} = 0.5+0.125 = 0.625$.  So we then have $2^{1025-1023}=2^2 = 4$ which gets multiplied by 1.652 to give: $4*1.625 = 6.5$.  Phew!

Notice, however, that not all decimal numbers can be represented exactly in binary (with e.g. 64 bits).  For example, consider the decimal number 0.3.  The closest number to 0.3 that can be represented with a 64-bit double is:

FIXME
\begin{align*}
0\,\,\,01111111101\,\,\,0011001100110011001100110011001100110011001100110011 &= +1 \times 2^{1021-1023} \times 1.0011001100110011...0011_2 \\
= 2^{-2}\times 1.2000000000000000\\ 
= 2.99999999999999988897769753748E-1
\end{align*}


See:  http://babbage.cs.qc.cuny.edu/IEEE-754.old/Decimal.html

In [21]:
print '{0:.31f}'.format(0.3)

0.2999999999999999888977697537484


In [33]:
0.2999999999999999888977697537484

0.3

## Adding floats
"Arithmetic among numbers in floating-point representation is not exact, even if the operands happen to be exactly represented (i.e., have exact values in the form of equation 1.1.1). For example, two floating numbers are added by first right-shifting (dividing by two) the mantissa of the smaller (in magnitude) one and simultaneously increasing its exponent until the two operands have the same exponent. Low-order (least significant) bits of the smaller operand are lost by this shifting. If the two operands differ too greatly in magnitude, then the smaller operand is effectively replaced by zero, since it is right-shifted to oblivion."

This is why, for a 64-bit double, there is no difference between 1.0 and (1.0+1e-30).  They both give 1.0 because the 1e-30 gets "bit-shifted to oblivion."
